In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3

In [ ]:
s3_client = boto3.client("s3")
sess = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
key_prefix = "ray_xgboost_distributed"

In [ ]:
!wget http://kt.ijs.si/elena_ikonomovska/datasets/airline/airline_14col.data.bz2

In [ ]:
input_data = sess.upload_data("airline_14col.data.bz2", bucket, key_prefix=f"{key_prefix}/input")

In [ ]:
subnets=None
security_group_ids=None

In [ ]:
# Pytorch Image is used to enable distributed GPU training
estimator_gpu = PyTorch(
    source_dir="src",
    entry_point="train_xgboost_airline.py",
    subnets=subnets,
    security_group_ids=security_group_ids,
    role=role,
    instance_count=4, 
    instance_type="ml.g5.8xlarge",
    framework_version="1.8",
    py_version="py36"
)

In [ ]:
estimator_gpu.fit({"train": input_data})